***

**Multiclass Classification**
    
**[AAA]**

**[AAA]**
    
**[AAA]**

***

In [2]:
!pip install shap

     |████████████████████████████████| 276kB 2.7MB/s 
  Created wheel for shap: filename=shap-0.35.0-cp36-cp36m-linux_x86_64.whl size=394141 sha256=40eb19f3e7a3a2bb2f939ef1366181a9d1e064d6b1019766d12ef2985572bb20
  Stored in directory: /root/.cache/pip/wheels/e7/f7/0f/b57055080cf8894906b3bd3616d2fc2bfd0b12d5161bcb24ac
Successfully built shap


In [4]:
!pip install category_encoders

     |████████████████████████████████| 102kB 2.3MB/s 


In [6]:
!pip install vecstack

  Created wheel for vecstack: filename=vecstack-0.4.0-cp36-none-any.whl size=19877 sha256=a4a1a35ea9563d8c118972e379de964f69eff70418b99d3e223e599248c8d511
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import missingno as mno
import warnings
warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')

import shap
import xgboost as xgb
import lightgbm as lgb

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split, \
                                    StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, \
                            average_precision_score, recall_score, \
                            precision_score, f1_score, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin

from category_encoders import OneHotEncoder, TargetEncoder, OrdinalEncoder

from pprint import pprint

from vecstack import StackingTransformer

RANDOM_STATE = 1805

### Loading data

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
xtrain = pd.read_csv('/content/drive/My Drive/HACKATHON DAUPHINE/Data/ref_train_x.csv', delimiter=',')
ytrain = pd.read_csv('/content/drive/My Drive/HACKATHON DAUPHINE/Data/ref_train_y.csv', delimiter=',')
testing_set = pd.read_csv('/content/drive/My Drive/HACKATHON DAUPHINE/Data/ref_test_x.csv', delimiter=',')

In [15]:
print(xtrain.shape)
print(ytrain.shape)

(15249, 23)
(15248, 1)


In [16]:
training_set = pd.concat([xtrain,ytrain], axis=1)
training_set.head()

,raw_id,sector,earnings_implied_obs,delta_vol_1w,delta_vol_1y,return_1w,return_1m,return_1y,implied_vol_3m,realised_vol_1w,realised_vol_1m,realised_vol_1y,ratio_put_call,publication_date_funda,exchange,net_income,shareholders_equity,net_debt,ebitda,ebit,sales,cash_flow,payout_ratio,0
0,2c1d2f99-b9c4-4541-8639-283ce4a20151,4700,3.7452,-0.3692,-0.0972,2.2375,1.1495,22.1040,17.1714,0.787627,0.884178,1.401695,0.516182,364.0,US,0.018193,1.011748,0.924447,0.306404,0.196460,1.000361,0.245411,265.517241,0.0
1,97049514-254d-4c3c-82c3-b1bb0a6117fe,4700,3.2026,-0.6008,0.6127,3.3098,1.3508,17.5971,15.9524,0.939068,0.833054,1.370815,0.540048,66.0,US,0.065461,0.998823,0.836746,0.280513,0.173050,0.950767,0.225520,72.325581,0.0
2,0f353f95-54d1-4a03-9099-06eea9981365,4700,4.4641,1.6247,-5.4056,-0.6884,-2.3511,6.2714,16.4591,0.959970,0.714396,0.907072,0.558759,255.0,US,0.065711,1.002628,0.839933,0.281582,0.173709,0.954389,0.226379,72.325581,0.0
3,b74cc12f-0582-415d-83c5-a8ee64783c60,4700,5.4850,0.3439,-0.4482,1.5906,5.5010,9.4912,15.9525,0.503026,0.593907,0.877211,0.798445,358.0,US,0.064607,0.985790,0.825828,0.276853,0.170792,0.938362,0.222578,72.325581,0.0
4,d68889cd-6018-436b-a6b3-558fe9dbc78a,4700,2.3555,0.4032,-1.2826,1.4269,4.4619,17.4692,14.4131,0.947279,0.994358,0.876525,0.693418,62.0,US,-0.111328,0.756207,0.759751,0.252945,0.154876,0.779753,0.197967,72.325581,0.0


In [0]:
print(training_set.shape)
print(testing_set.shape)
print(labels.shape)
print(results_example.shape)

(959506, 97)
(953897, 97)
(959506, 2)
(953897, 2)


In [24]:
training_set = training_set.rename(columns={"0": "labels"})
training_set.head()

,raw_id,sector,earnings_implied_obs,delta_vol_1w,delta_vol_1y,return_1w,return_1m,return_1y,implied_vol_3m,realised_vol_1w,realised_vol_1m,realised_vol_1y,ratio_put_call,publication_date_funda,exchange,net_income,shareholders_equity,net_debt,ebitda,ebit,sales,cash_flow,payout_ratio,labels
0,2c1d2f99-b9c4-4541-8639-283ce4a20151,4700,3.7452,-0.3692,-0.0972,2.2375,1.1495,22.1040,17.1714,0.787627,0.884178,1.401695,0.516182,364.0,US,0.018193,1.011748,0.924447,0.306404,0.196460,1.000361,0.245411,265.517241,0.0
1,97049514-254d-4c3c-82c3-b1bb0a6117fe,4700,3.2026,-0.6008,0.6127,3.3098,1.3508,17.5971,15.9524,0.939068,0.833054,1.370815,0.540048,66.0,US,0.065461,0.998823,0.836746,0.280513,0.173050,0.950767,0.225520,72.325581,0.0
2,0f353f95-54d1-4a03-9099-06eea9981365,4700,4.4641,1.6247,-5.4056,-0.6884,-2.3511,6.2714,16.4591,0.959970,0.714396,0.907072,0.558759,255.0,US,0.065711,1.002628,0.839933,0.281582,0.173709,0.954389,0.226379,72.325581,0.0
3,b74cc12f-0582-415d-83c5-a8ee64783c60,4700,5.4850,0.3439,-0.4482,1.5906,5.5010,9.4912,15.9525,0.503026,0.593907,0.877211,0.798445,358.0,US,0.064607,0.985790,0.825828,0.276853,0.170792,0.938362,0.222578,72.325581,0.0
4,d68889cd-6018-436b-a6b3-558fe9dbc78a,4700,2.3555,0.4032,-1.2826,1.4269,4.4619,17.4692,14.4131,0.947279,0.994358,0.876525,0.693418,62.0,US,-0.111328,0.756207,0.759751,0.252945,0.154876,0.779753,0.197967,72.325581,0.0


In [26]:
label = (training_set['labels'] == 1)
#print('ones = {}'.format( label.sum()[0]))

IndexError: ignored

For starting:

In [0]:
%%time
X = training_set.set_index('ID').copy()
y = labels.set_index('ID')
n_samples = len(y) // 16
X_resampled, y_resampled = resample(X, y, n_samples=n_samples, replace=False,
                                    stratify=y, random_state=RANDOM_STATE)

Wall time: 5.54 s


In [0]:
# Check shape
print(X_resampled.shape, y_resampled.shape)

# Check if same target variable distribution
print(y['source_id'].value_counts(True))
print(y_resampled['source_id'].value_counts(True))

(59969, 96) (59969, 1)
4    0.338139
2    0.265883
1    0.159674
3    0.125852
5    0.076179
0    0.034274
Name: source_id, dtype: float64
4    0.338141
2    0.265887
1    0.159683
3    0.125848
5    0.076173
0    0.034268
Name: source_id, dtype: float64


# Feature Engineering

## Categorical Variables

**Identify Categorical Encoders**

In [0]:
nunique_features = X_resampled.nunique()

In [0]:
nunique_features.sort_values().head(20)

(9, source_id)      6
(8, source_id)      6
(1, source_id)      6
(7, source_id)      6
(6, source_id)      6
(2, source_id)      6
(0, source_id)      6
(5, source_id)      6
(3, source_id)      6
(4, source_id)      6
day_id            122
(0, price)        268
(1, price)        273
(2, price)        295
(4, ask)          315
(3, price)        318
(4, bid)          327
(4, price)        332
(5, price)        357
(1, ask)          362
dtype: int64

In [0]:
categorical_features = nunique_features.sort_values().head(10).index
categorical_features = categorical_features.tolist()
pprint(sorted(categorical_features))

[(0, 'source_id'),
 (1, 'source_id'),
 (2, 'source_id'),
 (3, 'source_id'),
 (4, 'source_id'),
 (5, 'source_id'),
 (6, 'source_id'),
 (7, 'source_id'),
 (8, 'source_id'),
 (9, 'source_id')]


In [0]:
features_to_drop = ['day_id', 'stock_id']

In [0]:
numerical_features = [col for col in X_resampled.columns.tolist()
                      if col not in categorical_features + features_to_drop]
print(len(numerical_features))

84


**One-Hot Encoding**

In [0]:
%%time
oneHotTransformer = OneHotEncoder(cols=categorical_features)
oneHotTransformer.fit(X_resampled)

X_cat = oneHotTransformer.transform(X_resampled)
X_cat = X_cat.drop(features_to_drop, axis=1)

Wall time: 1.79 s


In [0]:
X_cat.shape

(59969, 154)

**Label Encoding**

In [0]:
%%time
categorical_features_plus = categorical_features + ['day_id', 'stock_id']
labelTransformer = OrdinalEncoder(cols=categorical_features_plus)
labelTransformer.fit(X_resampled)

X_cat_label = labelTransformer.transform(X_resampled)

Wall time: 1.9 s


In [0]:
X_cat_label.shape

(59969, 96)

## Missing Values

**Linear Models and Random Forests**

In [0]:
X_cat.isnull().sum().sort_values(ascending=False).head(40)

(2, bid1)              1138
(2, bid_size1)         1138
(2, ask1)               944
(2, ask_size1)          944
(2, ask)                 35
(2, ask_size)            35
(2, bid)                 35
(2, bid_size)            35
(2, ts_last_update)      34
(3, ask1)                 1
(3, ask)                  1
(3, ask_size)             1
(3, ask_size1)            1
(3, bid)                  1
(3, bid1)                 1
(0, ask_size1)            1
(0, ask_size)             1
(0, ask1)                 1
(0, ask)                  1
(3, bid_size)             1
(3, bid_size1)            1
(0, 'source_id')_4        0
(9, tod)                  0
(0, 'source_id')_-1       0
(0, 'source_id')_5        0
(3, 'source_id')_1        0
(4, 'source_id')_1        0
(3, 'source_id')_-1       0
(3, 'source_id')_6        0
(3, 'source_id')_5        0
(3, 'source_id')_4        0
(3, 'source_id')_3        0
(3, 'source_id')_2        0
(5, 'source_id')_-1       0
(0, 'source_id')_6        0
(5, 'source_id')_6  

In [0]:
imputer = SimpleImputer(strategy='median')
X_impute = imputer.fit_transform(X_cat)
X_impute = pd.DataFrame(X_impute, columns=X_cat.columns,
                        index=X_cat.index)

In [0]:
X_impute.isnull().sum().sum()

0

**XGBoost / LightGBM**

In [0]:
X_no_impute = X_cat_label.copy()

## Feature Scaling

**Linear Models**

In [0]:
X_scaled = X_impute.copy()

scaler = StandardScaler()
X_scaled[numerical_features] = scaler.fit_transform(X_impute[numerical_features])

**Random Forests / XGBoost / LightGBM**

In [0]:
X_rf = X_impute.copy()
X_boost = X_no_impute.copy()

# Model Training

In [0]:
print('Linear Models :\t', X_scaled.shape)
print('Random Forests:\t', X_rf.shape)
print('Gradient Boost:\t', X_boost.shape)

Linear Models :	 (59969, 154)
Random Forests:	 (59969, 154)
Gradient Boost:	 (59969, 96)


## Logistic Regression

In [0]:
%%time
logit = LogisticRegression(penalty='l2',
                           C=1.0,
                           solver='liblinear',
                           multi_class='ovr',
                           class_weight='balanced',
                           random_state=RANDOM_STATE)

cv_scores = cross_val_score(logit, X=X_scaled,
                            y=y_resampled,
                            cv=2, verbose=0,
                            scoring='accuracy')
print(cv_scores)

[0.3911492  0.38832005]
Wall time: 58.4 s


In [0]:
print('Mean CV  :\t', np.mean(cv_scores))
print('StDev CV :\t', np.std(cv_scores))

Mean CV  :	 0.3897346254942987
StDev CV :	 0.001414577467083289


In [0]:
%%time
_ = logit.fit(X_scaled, y_resampled)

Wall time: 1min 13s


## Linear SVM

In [0]:
%%time
linear_svc = LinearSVC(penalty='l2',
                       C=1.0,
                       multi_class='ovr',
                       class_weight='balanced',
                       random_state=RANDOM_STATE)

cv_scores = cross_val_score(linear_svc, X=X_scaled,
                            y=y_resampled,
                            cv=2, verbose=0,
                            scoring='accuracy')
print(cv_scores)

[0.39308344 0.3875863 ]
Wall time: 2min 49s


In [0]:
print('Mean CV  :\t', np.mean(cv_scores))
print('StDev CV :\t', np.std(cv_scores))

Mean CV  :	 0.39033486892044134
StDev CV :	 0.0027485700177298233


In [0]:
%%time
_ = linear_svc.fit(X_scaled, y_resampled)

Wall time: 2min 57s


## Random Forests

In [0]:
%%time
rf_model = RandomForestClassifier(n_estimators=200,
                                  max_depth=8,
                                  max_features='auto',
                                  class_weight='balanced',
                                  random_state=RANDOM_STATE)

cv_scores = cross_val_score(rf_model, X=X_rf,
                            y=y_resampled,
                            cv=2, verbose=0,
                            scoring='accuracy')
print(cv_scores)

[0.40365504 0.40516293]
Wall time: 37.2 s


In [0]:
print('Mean CV  :\t', np.mean(cv_scores))
print('StDev CV :\t', np.std(cv_scores))

Mean CV  :	 0.4044089823380407
StDev CV :	 0.0007539433198321677


In [0]:
%%time
_ = rf_model.fit(X_rf, y_resampled)

Wall time: 37.3 s


## XGBoost

**Cross-Validation Check**

In [0]:
# Custom Metric
def xgb_mean_accuracy(preds, dtrain):
    labels = dtrain.get_label()
    return 'm-accuracy', accuracy_score(labels, preds)

In [0]:
%%time
nb_classes = len(np.unique(y_resampled))

xgb_train = xgb.DMatrix(X_boost, label=y_resampled)

xgb_hyperparameters = {'booster': 'gbtree',
                       'tree_method': 'hist',
                       'eta': 0.1,
                       'colsample_bytree': 1.0,
                       'subsample': 1.0,
                       'objective': 'multi:softmax',
                       'max_depth': 8,
                       'alpha': 1.0,
                       'lambda': 1.0,
                       'num_class': nb_classes,
                       #'eval_metric': 'merror',
                       'disable_default_eval_metric': 1,
                       'seed': RANDOM_STATE
                      }

xgb_classifier_cv = xgb.cv(xgb_hyperparameters, xgb_train, nfold=2, maximize=True,
                           verbose_eval=10, num_boost_round=200, early_stopping_rounds=None,
                           feval=xgb_mean_accuracy)

[0]	train-m-accuracy:0.529306+0.0035595	train-merror:0.470693+0.0035595	test-m-accuracy:0.437326+0.003743	test-merror:0.562674+0.003743
[10]	train-m-accuracy:0.596409+0.0009105	train-merror:0.403591+0.0009105	test-m-accuracy:0.468726+0.0006245	test-merror:0.531274+0.0006245
[20]	train-m-accuracy:0.637696+5.6e-05	train-merror:0.362304+5.6e-05	test-m-accuracy:0.475645+0.0015085	test-merror:0.524355+0.0015085
[30]	train-m-accuracy:0.676767+0.0012285	train-merror:0.323233+0.0012285	test-m-accuracy:0.476313+0.0011085	test-merror:0.523688+0.0011085
[40]	train-m-accuracy:0.712485+0.001296	train-merror:0.287515+0.001296	test-m-accuracy:0.478564+0.000358	test-merror:0.521436+0.000358
[50]	train-m-accuracy:0.749104+0.000963	train-merror:0.250896+0.000963	test-m-accuracy:0.478864+0.000109	test-merror:0.521136+0.000109
[60]	train-m-accuracy:0.779453+0.000363	train-merror:0.220547+0.000363	test-m-accuracy:0.479398+0.000609	test-merror:0.520602+0.000609
[70]	train-m-accuracy:0.808167+0.0009035	train

**Fit Main Model**

In [0]:
class xgboostClassifier(BaseEstimator, ClassifierMixin):
    """ AAA
    """
    def __init__(self, xgb_params):
        """ AAA """
        self.xgb_params = xgb_params
        
    def fit(self, X, y, num_rounds=50, maximize=False):
        """ AAA """
        xgb_params = self.xgb_params
        xgb_train = xgb.DMatrix(X, label=y)
        clf = xgb.train(xgb_params, xgb_train, maximize=maximize,
                        verbose_eval=0, num_boost_round=num_rounds,
                        early_stopping_rounds=None)
        self.clf = clf
        return self
        
    def predict(self, X):
        """ AAA """
        clf = self.clf
        xgb_test = xgb.DMatrix(X)
        preds = clf.predict(xgb_test)
        return preds

In [0]:
%%time
nb_xgb_optimal = 100
xgb_classifier = xgboostClassifier(xgb_hyperparameters)
_ = xgb_classifier.fit(X_boost, y_resampled, num_rounds=nb_xgb_optimal,
                       maximize=False)

Wall time: 1min 39s


In [0]:
# y_pred = xgb_classifier.predict(X_boost)

## LightGBM

**Cross-Validation Check**

In [0]:
def lgb_mean_accuracy(preds, train_data):
    labels = train_data.get_label()
    return 'm-accuracy', accuracy_score(labels, preds), True

lgb_mean_accuracy_ = lambda preds, train_data: \
                    [lgb_mean_accuracy(preds, train_data)]

In [0]:
lgb_hyperparameters = {'objective': 'multiclass',
                       'boosting_type': 'gbdt',
                       'metric': 'multi_error',
                       'reg_lambda': 1.0,
                       'reg_alpha': 1.0,
                       'class_weight': 'is_unbalance',
                       'max_depth': 8,
                       'num_leaves': 150,
                       'feature_fraction': 0.95,
                       'learning_rate': 0.1,
                       'num_class': nb_classes,
                       'random_state': RANDOM_STATE
                       }

y_array = y_resampled['source_id'].values
lgb_train = lgb.Dataset(X_boost, label=y_array)

lgbm_classifier = lgb.cv(lgb_hyperparameters, lgb_train, num_boost_round=200,
                         early_stopping_rounds=None, nfold=2, verbose_eval=10)

[10]	cv_agg's multi_error: 0.530674 + 0.00342831
[20]	cv_agg's multi_error: 0.526039 + 0.00296117
[30]	cv_agg's multi_error: 0.523787 + 0.00311113
[40]	cv_agg's multi_error: 0.52237 + 0.00352794
[50]	cv_agg's multi_error: 0.522253 + 0.00337786
[60]	cv_agg's multi_error: 0.521003 + 0.00276081
[70]	cv_agg's multi_error: 0.520936 + 0.00342782
[80]	cv_agg's multi_error: 0.521136 + 0.00346118
[90]	cv_agg's multi_error: 0.521937 + 0.00422828
[100]	cv_agg's multi_error: 0.522237 + 0.00372804
[110]	cv_agg's multi_error: 0.522187 + 0.00377806
[120]	cv_agg's multi_error: 0.52167 + 0.00302765
[130]	cv_agg's multi_error: 0.522654 + 0.00264417
[140]	cv_agg's multi_error: 0.523471 + 0.00199387
[150]	cv_agg's multi_error: 0.523554 + 0.00257751
[160]	cv_agg's multi_error: 0.524171 + 0.00236076
[170]	cv_agg's multi_error: 0.524871 + 0.0024275
[180]	cv_agg's multi_error: 0.524521 + 0.00177714
[190]	cv_agg's multi_error: 0.524871 + 0.00109348
[200]	cv_agg's multi_error: 0.525472 + 0.00116021


**Fit Train Model**

In [0]:
class lightgbmClassifier(BaseEstimator, ClassifierMixin):
    """ AAA
    """
    def __init__(self, lgb_params):
        """ AAA """
        self.lgb_params = lgb_params
        
    def fit(self, X, y, num_rounds=50):
        """ AAA """
        lgb_params = self.lgb_params
        lgb_train = lgb.Dataset(X, label=y)
        clf = lgb.train(lgb_params, lgb_train, verbose_eval=0,
                        num_boost_round=num_rounds,
                        early_stopping_rounds=None)
        self.clf = clf
        return self
        
    def predict(self, X):
        """ AAA """
        preds = self.clf.predict(X)
        preds = np.argmax(preds, axis=1)
        return preds

In [0]:
%%time
nb_lgb_optimal = 100
y_array = y_resampled['source_id'].values
lgb_classifier = lightgbmClassifier(lgb_hyperparameters)
_ = lgb_classifier.fit(X_boost, y_array, num_rounds=nb_lgb_optimal)

Wall time: 23.8 s


# Testing Set

In [0]:
X_test = testing_set.set_index('ID').copy()

## Prepare Testing Set

**Linear Models**

In [0]:
%%time
X_test_one_hot = oneHotTransformer.transform(X_test)

Wall time: 18.2 s


In [0]:
X_test_one_hot = X_test_one_hot.drop(features_to_drop, axis=1)

In [0]:
%%time
X_test_impute = imputer.transform(X_test_one_hot)

Wall time: 2.27 s


In [0]:
X_test_impute = pd.DataFrame(X_test_impute, columns=X_test_one_hot.columns,
                             index=X_test_one_hot.index)

In [0]:
%%time
X_test_impute[numerical_features] = scaler.fit_transform(X_test_impute[numerical_features])

Wall time: 3.41 s


In [0]:
# Models
y_pred_test_logit = logit.predict(X_test_impute)
y_pred_test_svm = linear_svc.predict(X_test_impute)

**Random Forests**

In [0]:
%%time
X_test_one_hot_rf = oneHotTransformer.transform(X_test)

Wall time: 14 s


In [0]:
X_test_one_hot_rf = X_test_one_hot_rf.drop(features_to_drop, axis=1)

In [0]:
%%time
X_test_rf = imputer.transform(X_test_one_hot_rf)

Wall time: 2.15 s


In [0]:
X_test_rf = pd.DataFrame(X_test_rf, columns=X_test_one_hot_rf.columns,
                         index=X_test_one_hot_rf.index)

In [0]:
# Models
y_pred_test_rf = rf_model.predict(X_test_rf)

**XGBoost / LightGBM**

In [0]:
%%time
X_test_boost = labelTransformer.transform(X_test)

Wall time: 14 s


In [0]:
# Models
y_pred_test_xgb = xgb_classifier.predict(X_test_boost)
y_pred_test_lgb = lgb_classifier.predict(X_test_boost)

## Stacking

**Prediction Correlation**

In [0]:
test_preds = (y_pred_test_logit, y_pred_test_svm, y_pred_test_rf,
              y_pred_test_xgb, y_pred_test_lgb)

**Train Stacked Model**

https://www.kaggle.com/eliotbarr/stacking-test-sklearn-xgboost-catboost-lightgbm

In [0]:
def get_oof(clf, x_train, y_train, x_test, NFOLDS):
    
    ntrain = len(x_train)
    ntest = len(x_test)
    
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf.split(x_train)):
        x_tr = x_train.loc[train_index]
        y_tr = y_train.loc[train_index]
        x_te = x_train.loc[test_index]

        clf.fit(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [0]:
xg_oof_train, xg_oof_test = get_oof(xg)
et_oof_train, et_oof_test = get_oof(et)
rf_oof_train, rf_oof_test = get_oof(rf)
cb_oof_train, cb_oof_test = get_oof(cb)

x_train = np.concatenate((xg_oof_train, et_oof_train, rf_oof_train, cb_oof_train), axis=1)
x_test = np.concatenate((xg_oof_test, et_oof_test, rf_oof_test, cb_oof_test), axis=1)

logistic_regression = LogisticRegression()
logistic_regression.fit(x_train,y_train);

**Predicted**

In [0]:
logit_oof = get_oof()

## Predictions & Submits